In [34]:
import os
import pandas
import geopandas as gpd
from shapely.geometry import Point, MultiPolygon, Polygon

In [4]:
os.listdir()

['.ipynb_checkpoints',
 '2019_turisme_allotjament.gpkg',
 '2019_turisme_intensitat.gpkg',
 'hackaton.ipynb',
 'opendatabcn_pics-csv.csv',
 'shapefile_distrito_barcelona.gpkg',
 'zonas_culturales.csv']

In [5]:
input_file = '2019_turisme_intensitat.gpkg'
data_intensitat = gpd.read_file(input_file)

In [6]:
data = gpd.read_file(input_file)

In [9]:
data_intensitat.head()

,Id,gridcode,Shape_Leng,Shape_Area,geometry
0,1,2,199.109805,2663.569266,"MULTIPOLYGON (((431408.914 4588332.450, 431365..."
1,2,2,184.918046,2125.287662,"MULTIPOLYGON (((432388.914 4588022.450, 432344..."
2,3,2,248.574737,3964.175597,"MULTIPOLYGON (((431178.914 4587912.450, 431137..."
3,4,2,720.494818,37884.287859,"MULTIPOLYGON (((433068.914 4587822.450, 432978..."
4,5,2,373.067291,10035.408917,"MULTIPOLYGON (((430838.914 4587542.450, 430764..."


Gridcode contiene información acerca de la intensiva de la actividad turistíca. Siendo 2 intensidad media y 3 intensidad alta.

In [10]:
set(data_intensitat.gridcode)

{2, 3}

In [11]:
data_cultura = pd.read_csv('opendatabcn_pics-csv.csv', encoding='utf-16', sep=',')
data_cultura

,register_id,name,institution_id,institution_name,created,modified,addresses_roadtype_id,addresses_roadtype_name,addresses_road_id,addresses_road_name,...,values_attribute_id,values_category,values_attribute_name,values_value,values_outstanding,values_description,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y
0,﻿93334081340,Hotel d'Entitats de Can Guardiola,NaN,NaN,1993-11-30T00:00:00+01:00,2023-02-01T15:33:20.977138+01:00,NaN,NaN,95109,Cuba,...,100002.0,Informació d'interès,E-mail,obs-satandreu.benestar@gencat.net,True,NaN,432201.211020,4.586806e+06,41.429959,2.188553
1,﻿92086038702,Bar Marsella,NaN,NaN,1992-03-13T00:00:00+01:00,2022-09-17T04:48:47.605726+02:00,NaN,NaN,308505,C Sant Pau,...,20001.0,Telèfons,Tel.,934427263,True,NaN,430698.350786,4.581082e+06,41.378278,2.171224
2,﻿99400204922,Telefèric de Barcelona,NaN,NaN,2008-04-22T14:37:02+02:00,2021-04-22T14:26:50.786623+02:00,NaN,NaN,224904,Pg Joan Borbó C Barcelona,...,100003.0,Informació d'interès,Web,http://www.telefericodebarcelona.com/ca/,True,NaN,432085.569413,4.580511e+06,41.373250,2.187876
3,﻿92168177308,Universitat Politècnica de Catalunya,NaN,NaN,1985-12-17T00:00:00+01:00,2023-02-01T17:02:45.969422+01:00,NaN,NaN,169023,C Jordi Girona,...,100002.0,Informació d'interès,E-mail,informacio-upc@upc.es,True,NaN,426137.592862,4.582311e+06,41.388942,2.116538
4,﻿99400245589,Casa del Guarda - Espai d'Interpretació del P...,9.208603e+10,Consorci del Parc Natural de la Serra de Colls...,2010-04-20T13:55:00+02:00,2022-08-05T11:07:59.496877+02:00,NaN,NaN,243807,Camí del Pantà,...,100002.0,Informació d'interès,E-mail,ci@parccollserola.net,True,NaN,424567.087459,4.585287e+06,41.415598,2.097385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,﻿92086009459,Mercat Sarrià,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:08.375814+02:00,NaN,NaN,278007,Pg Reina Elisenda Montcada,...,100008.0,Informació d'interès,Instagram,https://www.instagram.com/mercat_de_sarria/,False,NaN,426499.788078,4.583517e+06,41.399836,2.120723
867,﻿92086009448,Mercat Santa Caterina,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:09:58.922902+02:00,NaN,NaN,132952,Av Francesc Cambó,...,100005.0,Informació d'interès,Facebook,https://www.facebook.com/MercatSantaCaterinaBcn/,False,NaN,431288.347866,4.581978e+06,41.386395,2.178177
868,﻿92086009455,Mercat Sants,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:03.721978+02:00,NaN,NaN,304209,C Sant Jordi,...,100005.0,Informació d'interès,Facebook,https://www.facebook.com/mercatdesants,False,NaN,427561.269367,4.580720e+06,41.374741,2.133755
869,﻿92086009454,Mercat Hostafrancs,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:07:58.387785+02:00,NaN,NaN,94809,Carrer de la Creu Coberta,...,100005.0,Informació d'interès,Facebook,https://www.facebook.com/MercatDHostafrancs,False,NaN,428406.282141,4.580759e+06,41.375162,2.143854


In [14]:
data_cultura.columns

Index(['register_id', 'name', 'institution_id', 'institution_name', 'created',
       'modified', 'addresses_roadtype_id', 'addresses_roadtype_name',
       'addresses_road_id', 'addresses_road_name',
       'addresses_start_street_number', 'addresses_end_street_number',
       'addresses_neighborhood_id', 'addresses_neighborhood_name',
       'addresses_district_id', 'addresses_district_name',
       'addresses_zip_code', 'addresses_town', 'addresses_main_address',
       'addresses_type', 'values_id', 'values_attribute_id', 'values_category',
       'values_attribute_name', 'values_value', 'values_outstanding',
       'values_description', 'geo_epgs_25831_x', 'geo_epgs_25831_y',
       'geo_epgs_4326_x', 'geo_epgs_4326_y'],
      dtype='object')

In [15]:
def check_points_in_area(_point, _area):
    """Checks if one of the points is in the area"""
    return _area.contains(_point)

In [16]:
def check_points_in_column(_point, _df):
    """Checks if one of the point is inside one area
    in a column that contains areas
    _point: a object of the type Point that contains one point
    returns: the id of the row where the points correspond
    if neither of the points is in any area, the function returns -1
    """
    for i in _df.index:
        if ( check_points_in_area(_point, _df.geometry[i]) ):
            return True
    return False
        

In [35]:
unsaturated_places = []

for i in data_cultura.index:
    point = Point( data_cultura.geo_epgs_4326_y[i], data_cultura.geo_epgs_4326_x[i] )
    id_found = check_points_in_column(point, data_intensitat[data_intensitat.gridcode == 3])
    
    if(not id_found):
        unsaturated_places.append(i)
        

In [37]:
data_cultura.iloc[unsaturated_places]

,register_id,name,institution_id,institution_name,created,modified,addresses_roadtype_id,addresses_roadtype_name,addresses_road_id,addresses_road_name,...,values_category,values_attribute_name,values_value,values_outstanding,values_description,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,n_distri
0,﻿93334081340,Hotel d'Entitats de Can Guardiola,NaN,NaN,1993-11-30T00:00:00+01:00,2023-02-01T15:33:20.977138+01:00,NaN,NaN,95109,Cuba,...,Informació d'interès,E-mail,obs-satandreu.benestar@gencat.net,True,NaN,432201.211020,4.586806e+06,41.429959,2.188553,Sant Andreu
1,﻿92086038702,Bar Marsella,NaN,NaN,1992-03-13T00:00:00+01:00,2022-09-17T04:48:47.605726+02:00,NaN,NaN,308505,C Sant Pau,...,Telèfons,Tel.,934427263,True,NaN,430698.350786,4.581082e+06,41.378278,2.171224,Ciutat Vella
2,﻿99400204922,Telefèric de Barcelona,NaN,NaN,2008-04-22T14:37:02+02:00,2021-04-22T14:26:50.786623+02:00,NaN,NaN,224904,Pg Joan Borbó C Barcelona,...,Informació d'interès,Web,http://www.telefericodebarcelona.com/ca/,True,NaN,432085.569413,4.580511e+06,41.373250,2.187876,Ciutat Vella
3,﻿92168177308,Universitat Politècnica de Catalunya,NaN,NaN,1985-12-17T00:00:00+01:00,2023-02-01T17:02:45.969422+01:00,NaN,NaN,169023,C Jordi Girona,...,Informació d'interès,E-mail,informacio-upc@upc.es,True,NaN,426137.592862,4.582311e+06,41.388942,2.116538,Les Corts
4,﻿99400245589,Casa del Guarda - Espai d'Interpretació del P...,9.208603e+10,Consorci del Parc Natural de la Serra de Colls...,2010-04-20T13:55:00+02:00,2022-08-05T11:07:59.496877+02:00,NaN,NaN,243807,Camí del Pantà,...,Informació d'interès,E-mail,ci@parccollserola.net,True,NaN,424567.087459,4.585287e+06,41.415598,2.097385,SarriÃ -Sant Gervasi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,﻿92086009459,Mercat Sarrià,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:08.375814+02:00,NaN,NaN,278007,Pg Reina Elisenda Montcada,...,Informació d'interès,Instagram,https://www.instagram.com/mercat_de_sarria/,False,NaN,426499.788078,4.583517e+06,41.399836,2.120723,SarriÃ -Sant Gervasi
867,﻿92086009448,Mercat Santa Caterina,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:09:58.922902+02:00,NaN,NaN,132952,Av Francesc Cambó,...,Informació d'interès,Facebook,https://www.facebook.com/MercatSantaCaterinaBcn/,False,NaN,431288.347866,4.581978e+06,41.386395,2.178177,Ciutat Vella
868,﻿92086009455,Mercat Sants,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:03.721978+02:00,NaN,NaN,304209,C Sant Jordi,...,Informació d'interès,Facebook,https://www.facebook.com/mercatdesants,False,NaN,427561.269367,4.580720e+06,41.374741,2.133755,Sants-MontjuÃ¯c
869,﻿92086009454,Mercat Hostafrancs,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:07:58.387785+02:00,NaN,NaN,94809,Carrer de la Creu Coberta,...,Informació d'interès,Facebook,https://www.facebook.com/MercatDHostafrancs,False,NaN,428406.282141,4.580759e+06,41.375162,2.143854,Sants-MontjuÃ¯c


In [19]:
def check_points_in_barrio(_point , _df):
    """Checks if one of the point is inside one area
    in a column that contains areas
    _point: a object of the type Point that contains one point
    returns: the id of the row where the points correspond
    if neither of the points is in any area, the function returns -1
    """
    for i in _df.index:
        if ( check_points_in_area(_point, _df.geometry[i]) ):
            return _df.n_distri[i]

In [20]:
#Datos sobre los barrios y las cordenadas de su perimetro
input_file_2 = 'shapefile_distrito_barcelona.gpkg'
data_barrios = gpd.read_file(input_file_2, encoding='latin-1')

In [38]:
data_barrios.n_distri

0    SarriÃ -Sant Gervasi
1         Horta-GuinardÃ³
2            Ciutat Vella
3                Eixample
4         Sants-MontjuÃ¯c
5               Les Corts
6                 GrÃ cia
7              Nou Barris
8             Sant Andreu
9             Sant MartÃ­
Name: n_distri, dtype: object

In [39]:
name_distri = []

for i in data_cultura.index:
    
    point = Point( data_cultura.geo_epgs_4326_y[i], data_cultura.geo_epgs_4326_x[i] )
    barrio_name = check_points_in_barrio(point, data_barrios)
    name_distri.append(barrio_name)

In [40]:
name_distri

['Sant Andreu',
 'Ciutat Vella',
 'Ciutat Vella',
 'Les Corts',
 'SarriÃ\xa0-Sant Gervasi',
 'Sant Andreu',
 'Ciutat Vella',
 'Sant MartÃ\xad',
 'Ciutat Vella',
 'Les Corts',
 'Horta-GuinardÃ³',
 'SarriÃ\xa0-Sant Gervasi',
 'SarriÃ\xa0-Sant Gervasi',
 'Sant Andreu',
 'Sants-MontjuÃ¯c',
 'GrÃ\xa0cia',
 'GrÃ\xa0cia',
 'Sants-MontjuÃ¯c',
 'Eixample',
 'Eixample',
 'Les Corts',
 'Sants-MontjuÃ¯c',
 'Nou Barris',
 'Sant MartÃ\xad',
 'GrÃ\xa0cia',
 'Ciutat Vella',
 'Sant Andreu',
 'Sants-MontjuÃ¯c',
 'Les Corts',
 'Ciutat Vella',
 'Sant MartÃ\xad',
 'Horta-GuinardÃ³',
 'Ciutat Vella',
 'SarriÃ\xa0-Sant Gervasi',
 'Les Corts',
 'Sant MartÃ\xad',
 'GrÃ\xa0cia',
 'SarriÃ\xa0-Sant Gervasi',
 'Ciutat Vella',
 'Sant Andreu',
 'Sants-MontjuÃ¯c',
 'Sants-MontjuÃ¯c',
 'Eixample',
 'Sants-MontjuÃ¯c',
 'Sants-MontjuÃ¯c',
 'SarriÃ\xa0-Sant Gervasi',
 'Eixample',
 'GrÃ\xa0cia',
 'Sants-MontjuÃ¯c',
 'Les Corts',
 'Horta-GuinardÃ³',
 'GrÃ\xa0cia',
 'Horta-GuinardÃ³',
 'SarriÃ\xa0-Sant Gervasi',
 'Eixample'

In [41]:
data_barrios.head()

,cartodb_id,c_distri,n_distri,homes,dones,area,perim,coord_x,coord_y,web_1,web_2,web_3,geometry
0,5,05,SarriÃ -Sant Gervasi,67799,80113,2.009280e+07,37563.642237,425388.507986,4.585170e+06,http://www.bcn.cat/sarria-santgervasi,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,"MULTIPOLYGON (((2.07327 41.41383, 2.07165 41.4..."
1,7,07,Horta-GuinardÃ³,79017,89075,1.194708e+07,20413.187364,429117.618770,4.586950e+06,http://www.bcn.cat/horta-guinardo,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,"POLYGON ((2.17731 41.42656, 2.17769 41.42612, ..."
2,1,01,Ciutat Vella,53968,48379,4.368465e+06,21035.207261,431616.773990,4.581564e+06,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,"POLYGON ((2.18354 41.39255, 2.18461 41.39174, ..."
3,2,02,Eixample,123571,142906,7.476392e+06,13902.573980,430243.353657,4.582773e+06,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,"POLYGON ((2.18354 41.39255, 2.18249 41.39335, ..."
4,3,03,Sants-MontjuÃ¯c,87877,95243,2.294042e+07,47125.925905,428562.773279,4.578163e+06,http://www.bcn.cat/sants-montjuic,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,"POLYGON ((2.16900 41.37611, 2.16971 41.37608, ..."


In [42]:
data_cultura.head()

,register_id,name,institution_id,institution_name,created,modified,addresses_roadtype_id,addresses_roadtype_name,addresses_road_id,addresses_road_name,...,values_category,values_attribute_name,values_value,values_outstanding,values_description,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,n_distri
0,﻿93334081340,Hotel d'Entitats de Can Guardiola,NaN,NaN,1993-11-30T00:00:00+01:00,2023-02-01T15:33:20.977138+01:00,NaN,NaN,95109,Cuba,...,Informació d'interès,E-mail,obs-satandreu.benestar@gencat.net,True,NaN,432201.211020,4.586806e+06,41.429959,2.188553,Sant Andreu
1,﻿92086038702,Bar Marsella,NaN,NaN,1992-03-13T00:00:00+01:00,2022-09-17T04:48:47.605726+02:00,NaN,NaN,308505,C Sant Pau,...,Telèfons,Tel.,934427263,True,NaN,430698.350786,4.581082e+06,41.378278,2.171224,Ciutat Vella
2,﻿99400204922,Telefèric de Barcelona,NaN,NaN,2008-04-22T14:37:02+02:00,2021-04-22T14:26:50.786623+02:00,NaN,NaN,224904,Pg Joan Borbó C Barcelona,...,Informació d'interès,Web,http://www.telefericodebarcelona.com/ca/,True,NaN,432085.569413,4.580511e+06,41.373250,2.187876,Ciutat Vella
3,﻿92168177308,Universitat Politècnica de Catalunya,NaN,NaN,1985-12-17T00:00:00+01:00,2023-02-01T17:02:45.969422+01:00,NaN,NaN,169023,C Jordi Girona,...,Informació d'interès,E-mail,informacio-upc@upc.es,True,NaN,426137.592862,4.582311e+06,41.388942,2.116538,Les Corts
4,﻿99400245589,Casa del Guarda - Espai d'Interpretació del P...,9.208603e+10,Consorci del Parc Natural de la Serra de Colls...,2010-04-20T13:55:00+02:00,2022-08-05T11:07:59.496877+02:00,NaN,NaN,243807,Camí del Pantà,...,Informació d'interès,E-mail,ci@parccollserola.net,True,NaN,424567.087459,4.585287e+06,41.415598,2.097385,SarriÃ -Sant Gervasi


In [26]:
data_cultura["n_distri"] = name_distri

In [27]:
data_cultura

,register_id,name,institution_id,institution_name,created,modified,addresses_roadtype_id,addresses_roadtype_name,addresses_road_id,addresses_road_name,...,values_category,values_attribute_name,values_value,values_outstanding,values_description,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,n_distri
0,﻿93334081340,Hotel d'Entitats de Can Guardiola,NaN,NaN,1993-11-30T00:00:00+01:00,2023-02-01T15:33:20.977138+01:00,NaN,NaN,95109,Cuba,...,Informació d'interès,E-mail,obs-satandreu.benestar@gencat.net,True,NaN,432201.211020,4.586806e+06,41.429959,2.188553,Sant Andreu
1,﻿92086038702,Bar Marsella,NaN,NaN,1992-03-13T00:00:00+01:00,2022-09-17T04:48:47.605726+02:00,NaN,NaN,308505,C Sant Pau,...,Telèfons,Tel.,934427263,True,NaN,430698.350786,4.581082e+06,41.378278,2.171224,Ciutat Vella
2,﻿99400204922,Telefèric de Barcelona,NaN,NaN,2008-04-22T14:37:02+02:00,2021-04-22T14:26:50.786623+02:00,NaN,NaN,224904,Pg Joan Borbó C Barcelona,...,Informació d'interès,Web,http://www.telefericodebarcelona.com/ca/,True,NaN,432085.569413,4.580511e+06,41.373250,2.187876,Ciutat Vella
3,﻿92168177308,Universitat Politècnica de Catalunya,NaN,NaN,1985-12-17T00:00:00+01:00,2023-02-01T17:02:45.969422+01:00,NaN,NaN,169023,C Jordi Girona,...,Informació d'interès,E-mail,informacio-upc@upc.es,True,NaN,426137.592862,4.582311e+06,41.388942,2.116538,Les Corts
4,﻿99400245589,Casa del Guarda - Espai d'Interpretació del P...,9.208603e+10,Consorci del Parc Natural de la Serra de Colls...,2010-04-20T13:55:00+02:00,2022-08-05T11:07:59.496877+02:00,NaN,NaN,243807,Camí del Pantà,...,Informació d'interès,E-mail,ci@parccollserola.net,True,NaN,424567.087459,4.585287e+06,41.415598,2.097385,SarriÃ -Sant Gervasi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,﻿92086009459,Mercat Sarrià,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:08.375814+02:00,NaN,NaN,278007,Pg Reina Elisenda Montcada,...,Informació d'interès,Instagram,https://www.instagram.com/mercat_de_sarria/,False,NaN,426499.788078,4.583517e+06,41.399836,2.120723,SarriÃ -Sant Gervasi
867,﻿92086009448,Mercat Santa Caterina,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:09:58.922902+02:00,NaN,NaN,132952,Av Francesc Cambó,...,Informació d'interès,Facebook,https://www.facebook.com/MercatSantaCaterinaBcn/,False,NaN,431288.347866,4.581978e+06,41.386395,2.178177,Ciutat Vella
868,﻿92086009455,Mercat Sants,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:03.721978+02:00,NaN,NaN,304209,C Sant Jordi,...,Informació d'interès,Facebook,https://www.facebook.com/mercatdesants,False,NaN,427561.269367,4.580720e+06,41.374741,2.133755,Sants-MontjuÃ¯c
869,﻿92086009454,Mercat Hostafrancs,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:07:58.387785+02:00,NaN,NaN,94809,Carrer de la Creu Coberta,...,Informació d'interès,Facebook,https://www.facebook.com/MercatDHostafrancs,False,NaN,428406.282141,4.580759e+06,41.375162,2.143854,Sants-MontjuÃ¯c


In [28]:
data_cultura.iloc[unsaturated_places].reset_index()

,index,register_id,name,institution_id,institution_name,created,modified,addresses_roadtype_id,addresses_roadtype_name,addresses_road_id,...,values_category,values_attribute_name,values_value,values_outstanding,values_description,geo_epgs_25831_x,geo_epgs_25831_y,geo_epgs_4326_x,geo_epgs_4326_y,n_distri
0,0,﻿93334081340,Hotel d'Entitats de Can Guardiola,NaN,NaN,1993-11-30T00:00:00+01:00,2023-02-01T15:33:20.977138+01:00,NaN,NaN,95109,...,Informació d'interès,E-mail,obs-satandreu.benestar@gencat.net,True,NaN,432201.211020,4.586806e+06,41.429959,2.188553,Sant Andreu
1,1,﻿92086038702,Bar Marsella,NaN,NaN,1992-03-13T00:00:00+01:00,2022-09-17T04:48:47.605726+02:00,NaN,NaN,308505,...,Telèfons,Tel.,934427263,True,NaN,430698.350786,4.581082e+06,41.378278,2.171224,Ciutat Vella
2,2,﻿99400204922,Telefèric de Barcelona,NaN,NaN,2008-04-22T14:37:02+02:00,2021-04-22T14:26:50.786623+02:00,NaN,NaN,224904,...,Informació d'interès,Web,http://www.telefericodebarcelona.com/ca/,True,NaN,432085.569413,4.580511e+06,41.373250,2.187876,Ciutat Vella
3,3,﻿92168177308,Universitat Politècnica de Catalunya,NaN,NaN,1985-12-17T00:00:00+01:00,2023-02-01T17:02:45.969422+01:00,NaN,NaN,169023,...,Informació d'interès,E-mail,informacio-upc@upc.es,True,NaN,426137.592862,4.582311e+06,41.388942,2.116538,Les Corts
4,4,﻿99400245589,Casa del Guarda - Espai d'Interpretació del P...,9.208603e+10,Consorci del Parc Natural de la Serra de Colls...,2010-04-20T13:55:00+02:00,2022-08-05T11:07:59.496877+02:00,NaN,NaN,243807,...,Informació d'interès,E-mail,ci@parccollserola.net,True,NaN,424567.087459,4.585287e+06,41.415598,2.097385,SarriÃ -Sant Gervasi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,866,﻿92086009459,Mercat Sarrià,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:08.375814+02:00,NaN,NaN,278007,...,Informació d'interès,Instagram,https://www.instagram.com/mercat_de_sarria/,False,NaN,426499.788078,4.583517e+06,41.399836,2.120723,SarriÃ -Sant Gervasi
867,867,﻿92086009448,Mercat Santa Caterina,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:09:58.922902+02:00,NaN,NaN,132952,...,Informació d'interès,Facebook,https://www.facebook.com/MercatSantaCaterinaBcn/,False,NaN,431288.347866,4.581978e+06,41.386395,2.178177,Ciutat Vella
868,868,﻿92086009455,Mercat Sants,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:10:03.721978+02:00,NaN,NaN,304209,...,Informació d'interès,Facebook,https://www.facebook.com/mercatdesants,False,NaN,427561.269367,4.580720e+06,41.374741,2.133755,Sants-MontjuÃ¯c
869,869,﻿92086009454,Mercat Hostafrancs,NaN,NaN,1984-12-28T00:00:00+01:00,2023-06-15T11:07:58.387785+02:00,NaN,NaN,94809,...,Informació d'interès,Facebook,https://www.facebook.com/MercatDHostafrancs,False,NaN,428406.282141,4.580759e+06,41.375162,2.143854,Sants-MontjuÃ¯c


In [29]:
 table_result = data_cultura.groupby('n_distri').count().rename(columns={'name':'Count'})[['Count']]

In [30]:
data_cultura.to_csv("zonas_culturales.csv")

In [45]:
table_result = table_result.sort_values('Count', ascending=False)

In [32]:
list(data_cultura.groupby('n_distri') )

[('Ciutat Vella',
        register_id                                     name  institution_id   
  1    ﻿92086038702                             Bar Marsella             NaN  \
  2    ﻿99400204922                   Telefèric de Barcelona             NaN   
  6    ﻿96067083304          Casa - Fàbrica Erasme de Gònima             NaN   
  8    ﻿99400265020   TAMAIA Viure Sense Violencia, S.C.C.L.             NaN   
  25   ﻿92086000492  Reial Acadèmia de Medicina de Catalunya             NaN   
  ..            ...                                      ...             ...   
  834  ﻿75990024770               Museu de Cera de Barcelona             NaN   
  850  ﻿99400697947                            Pla de la Seu             NaN   
  862  ﻿92086009445                       Mercat Barceloneta             NaN   
  864  ﻿92086009447             Mercat Boqueria - Sant Josep             NaN   
  867  ﻿92086009448                    Mercat Santa Caterina             NaN   
  
      institution_n

In [49]:
zonas_culturales_subset = data_cultura[['n_distri', 'name']]
zonas_culturales_subset

,n_distri,name
0,Sant Andreu,Hotel d'Entitats de Can Guardiola
1,Ciutat Vella,Bar Marsella
2,Ciutat Vella,Telefèric de Barcelona
3,Les Corts,Universitat Politècnica de Catalunya
4,SarriÃ -Sant Gervasi,Casa del Guarda - Espai d'Interpretació del P...
...,...,...
866,SarriÃ -Sant Gervasi,Mercat Sarrià
867,Ciutat Vella,Mercat Santa Caterina
868,Sants-MontjuÃ¯c,Mercat Sants
869,Sants-MontjuÃ¯c,Mercat Hostafrancs


In [50]:
conteo_distritos = zonas_culturales_subset['n_distri'].value_counts(ascending=False)
conteo_distritos

n_distri
SarriÃ -Sant Gervasi    132
Ciutat Vella            124
GrÃ cia                 119
Sants-MontjuÃ¯c          95
Sant Andreu              91
Eixample                 88
Les Corts                70
Sant MartÃ­              58
Horta-GuinardÃ³          53
Nou Barris               30
Name: count, dtype: int64

In [51]:
zonas_culturales_subset.shape

(871, 2)

In [52]:
zonas_culturales_subset = zonas_culturales_subset[~zonas_culturales_subset['n_distri'].isin(['Ciutat Vella', 'Eixample'])]

In [53]:
zonas_culturales_subset

,n_distri,name
0,Sant Andreu,Hotel d'Entitats de Can Guardiola
3,Les Corts,Universitat Politècnica de Catalunya
4,SarriÃ -Sant Gervasi,Casa del Guarda - Espai d'Interpretació del P...
5,Sant Andreu,Torre del Fang
7,Sant MartÃ­,Platja de la Nova Mar Bella
...,...,...
865,SarriÃ -Sant Gervasi,Mercat Llibertat
866,SarriÃ -Sant Gervasi,Mercat Sarrià
868,Sants-MontjuÃ¯c,Mercat Sants
869,Sants-MontjuÃ¯c,Mercat Hostafrancs


In [54]:
zonas_culturales_puntos_turisticos = zonas_culturales_subset[zonas_culturales_subset['n_distri'].isin(['SarriÃƒÂ -Sant Gervasi',  'GrÃƒÂ cia', 'Sants-MontjuÃƒÂ¯c' ,  'Sant Andreu', 'Les Corts'])]
zonas_culturales_puntos_turisticos

,n_distri,name
0,Sant Andreu,Hotel d'Entitats de Can Guardiola
3,Les Corts,Universitat Politècnica de Catalunya
5,Sant Andreu,Torre del Fang
9,Les Corts,Museu Barça Immersive Tour
13,Sant Andreu,Can Fontanet - Centre d'Interpretació dels Tre...
...,...,...
841,Sant Andreu,Avenç Obrer Català de Sant Andreu
842,Sant Andreu,Carrer Pons i Gallarza
843,Sant Andreu,Carrer Puerto Príncipe
849,Les Corts,Mercat Les Corts
